In [88]:
import pandas as pd
import numpy as np
from scipy import stats
import os
import matplotlib.pyplot as plt

UF = "SP"

#dados dos candidatos
path_consultacand = "consulta_cand_2014/consulta_cand_2014_%s.txt" % UF

#bens dos candidatos
path_bemcand = "bem_candidato_2014/bem_candidato_2014_%s.txt" % UF

#prestacao de contas dos candidatos
path_prestacao = "prestacao_final_2014/despesas_candidatos_2014_%s.txt"  % UF

path_votacao = "votacao_candidato_munzona_2014/votacao_candidato_munzona_2014_%s.txt" % UF

cargo_filt = "Deputado Estadual"

In [89]:
#carregar os dados dos candidatos
names_cand = ["data", "hora", "ano", "turno", "desc", "uf", "ue", "estado", "cod_cargo", "cargo", "nome",
              "seq", "id", "cpf", "nome_urna", "cod_situ", "desc_situ", "num_partido",
             "sigla_partido", "nome_partido", "codigo_legenda", "sigla_legenda", "composicao_legenda",
             "nome_legenda", "codigo_ocupacao", "desc_ocupacao", "data_nasc", "titulo_eleitoral",
             "idade", "cod_sexo", "desc_sexo", "cod_instrucao", "instrucao", "cod_civil", "civil",
             "cor_cor", "cor", "cod_nacional", "nacional", "sigla_uf_natal"
              , "cod_mun_natal", "mun_natal", "despesa_maxima", "cod_sit_tot", "desc_sit_tot",
             "email"]
dat_cand = pd.read_csv(path_consultacand, delimiter=';', encoding="latin-1", names=names_cand)

dat_cand = dat_cand[dat_cand.cargo == cargo_filt.upper()]

In [90]:
#carregar os bens dos candidatos
names_bem = ["data", "hora", "ano", "desc", "uf", "seq", "cod_bem",
             "desc_bem", "detalhe_bem", "valor", "data_atu", "hora_atu"]
dat_bens = pd.read_csv(path_bemcand, delimiter=';', encoding="latin-1", names=names_bem)

In [91]:
#fazer uma lista dos bens totais dos candidaos
candidatos = np.unique(dat_cand.seq)
bens = np.zeros(len(candidatos))
seqs = np.zeros(len(candidatos))
i = 0
for candidato in candidatos:
    seqs[i] = candidato
    bens[i] = np.sum(dat_bens[dat_bens.seq == candidato].valor)
    i += 1

In [92]:
#carregar a prestacao dos candidatos
names_prestacao = ["cod_eleicao", "desc_eleicao", "data_hora", "cnpj_prestador", "seq", "uf",
                   "sigla_partido", "id", "cargo", "nome", "cpf", "tipo_doc", "num_doc",
                  "id_fornecedor", "nome_fornecedor","nome_fornecedor_receita" ,"cod_se_fornecedor",
                  "setor_fornecedor", "data_despesa", "valor", "tipo", "descricao"]
dat_prestacao = pd.read_csv(path_prestacao, header=0, delimiter=";", encoding="latin-1",
                            names = names_prestacao, decimal=",")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [93]:
candidatos = np.unique(dat_cand.seq)
prestacoes = np.zeros(len(candidatos))
seqs = np.zeros(len(candidatos))
i = 0
for candidato in candidatos:
    seqs[i] = candidato
    prestacoes[i] = np.sum(dat_prestacao[dat_prestacao.seq == candidato].valor)
    i += 1

In [94]:
names_votacao = ["data", "hora", "ano", "turno", "desc", "uf", "ue", "cod_mun", "nome_mun", "zona",
                 "cod_cargo", "num_cand", "seq", "nome", "nome_urna", "cargo", "cod_situ_s", "desc_situ_s",
                "cod_situ", "desc_situ", "cod_tot", "desc_tot", "num_partido", "sigla_partido", 
                "nome_partido", "seq_partido", "coligacao", "legenda", "votos", "transito"]
dat_voto = pd.read_csv(path_votacao, delimiter=';', encoding="latin-1", names=names_votacao)

In [95]:
candidatos = np.unique(dat_cand.seq)
votos = np.zeros(len(candidatos))
seqs = np.zeros(len(candidatos))
i = 0
for candidato in candidatos:
    seqs[i] = candidato
    votos[i] = np.sum(dat_voto[dat_voto.seq == candidato].votos)
    i += 1

In [99]:
tipos = np.unique(dat_prestacao.tipo)
valor_tip = []
for tip in tipos:
    valor_tip.append(np.sum(dat_prestacao[dat_prestacao.tipo == tip].valor))    

In [100]:
valor_tip = np.array(valor_tip)
#valor_tip = valor_tip / np.sum(valor_tip)
tab = np.vstack([tipos, valor_tip]).T
tab = pd.DataFrame(tab, columns=["tipo", "valor"]).sort_values("valor", ascending=False)
tab.to_csv("abs_tipo-gastos_dep-est_SP.csv")

In [9]:
(x, y) = (prestacoes, votos)
filt = (x != 0) & (y != 0)
x = np.log10(np.array(x[filt]))
y = np.log10(np.array(y[filt]))
fit = np.polyfit(x, y, deg=1, cov=True)

In [10]:
p0 = fit[0][0]
p1 = fit[0][1]

s0 = (fit[1][0,0])
s1 = (fit[1][1,1])

In [11]:
#plt.hist2d(x, y, bins=30)
#plt.colorbar()
plt.scatter(x, y)
#plt.plot(x, (p0 + s0) * x + (p1+s1), 'k--')
plt.plot(x, p0 * x + p1, 'r-')
#plt.plot(x, (p0 - s0) * x + (p1-s1), 'k--')
plt.title("Votos obtidos nas eleições vs prestação de contas (%s - %s)" % (cargo_filt, UF))
plt.xlabel("Prestações das contas - log10[x] - R$")
plt.ylabel("Quantidade de votos - log10[x]")
plt.show()

In [ ]:
 (2 ** 1/p0)

#deputado estadual SP
p0 = 0.67655917254413789
p1 = 0.42138786401850464

#deputado federal SP
p0 = 0.69529230273514808
p1 = 0.40946995135512709

#deputado estadual RO

#deputado federal RO
p0 = 0.60804182639112947
p1 = 0.67582003213090802